# Baseline Unet

In [1]:
import sys
import json
import pathlib

In [2]:
def get_root_dir(cwd: pathlib.Path = pathlib.Path().resolve(), anchor="README.md") -> pathlib.Path:
    """
    Get the root directory of the project by searching for a specific anchor file.
    i.e. find the root directory where anchor file README.md/.git is located.

    Args:
        cwd (pathlib.Path): Current working directory.
        anchor (str): The name of the anchor file to search for.

    Returns:
        pathlib.Path: The root directory of the project.

    Raises:
        FileNotFoundError: If the anchor file is not found in any parent directories.
    """
    # Check if the anchor file exists in the current working directory
    # If it does, return the current working directory
    # If it doesn't, check the parent directories until the anchor file is found
    if cwd.joinpath(anchor).exists():
        return cwd
    else:
        for parent in cwd.parents:
            if (parent / anchor).exists():
                return parent

    # If the anchor file is not found in any parent directories, raise an error
    raise FileNotFoundError(f"Anchor file '{anchor}' not found in any parent directories of {cwd}.")

In [3]:
# Git repository information
REPO_GIT_OWNER = "bennylao"
REPO_NAME = "cv-cam-based-img-segmentation"


### Logics to set up paths based on the environment (Google Colab or local machine) ###
COLAB_ROOT_PATH = pathlib.Path("/content")
IS_COLAB = COLAB_ROOT_PATH.exists()

if IS_COLAB:
    # Working on Google Colab
    from google.colab import drive

    # Mount Google Drive
    DRIVE_PATH = COLAB_ROOT_PATH.joinpath("drive")
    drive.flush_and_unmount()
    drive.mount(str(DRIVE_PATH))

    # Load git credentials from Google Drive
    DRIVE_FOLDER_PATH = DRIVE_PATH.joinpath("MyDrive", "Colab Notebooks")
    if DRIVE_FOLDER_PATH.exists():
        with open(DRIVE_FOLDER_PATH.joinpath("git_credentials.json"), "r") as f:
            git_config = json.load(f)
    else:
        raise FileNotFoundError(f"Config file not found at {DRIVE_FOLDER_PATH}")

    # Set up Git credentials
    GIT_USER_NAME = git_config["GIT_USER_NAME"]
    GIT_TOKEN = git_config["GIT_TOKEN"]
    GIT_USER_EMAIL = git_config["GIT_USER_EMAIL"]

    !git config --global user.email {GIT_USER_EMAIL}
    !git config --global user.name {GIT_USER_NAME}

    # Set up project paths
    CURRENT_PATH = pathlib.Path().resolve()
    ROOT = COLAB_ROOT_PATH.joinpath(REPO_NAME)
    DATA_DIR = ROOT.joinpath("data")
    MODEL_DIR = DRIVE_FOLDER_PATH.joinpath("models")
    OUTPUT_DIR = DRIVE_FOLDER_PATH.joinpath("output")

    # Clone repo
    GIT_PATH = f"https://{GIT_TOKEN}@github.com/{REPO_GIT_OWNER}/{REPO_NAME}.git"

    if not ROOT.exists():
        !git clone --depth 1 "{GIT_PATH}" "{ROOT}"
    else:
        print(f"Git repo already cloned at {ROOT}")
        !git -C "{ROOT}" pull

else:
    # Working on local machine
    CURRENT_PATH = pathlib.Path().resolve()
    ROOT = get_root_dir(CURRENT_PATH, anchor="README.md")
    DATA_DIR = ROOT.joinpath("data")
    MODEL_DIR = ROOT.joinpath("models")
    OUTPUT_DIR = ROOT.joinpath("output")

# Create folder if not exist
if not DATA_DIR.exists():
    DATA_DIR.mkdir(parents=True, exist_ok=True)
    print(f"Created data directory at {DATA_DIR}")

if not OUTPUT_DIR.exists():
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    print(f"Created output directory at {OUTPUT_DIR}")

if not MODEL_DIR.exists():
    MODEL_DIR.mkdir(parents=True, exist_ok=True)
    print(f"Created model directory at {MODEL_DIR}")

# Add root path to sys.path
sys.path.append(str(ROOT))

print("=" * 50)
print(f"Runtime: {'Google Colab' if IS_COLAB else 'Local Machine'}")
print(f"{CURRENT_PATH=}")
print(f"{ROOT=}")
print(f"{DATA_DIR=}")
print(f"{MODEL_DIR=}")
print(f"{OUTPUT_DIR=}")
print("=" * 50)

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive
Cloning into '/content/cv-cam-based-img-segmentation'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 31 (delta 2), reused 25 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (31/31), 49.56 KiB | 16.52 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Runtime: Google Colab
CURRENT_PATH=PosixPath('/content')
ROOT=PosixPath('/content/cv-cam-based-img-segmentation')
DATA_DIR=PosixPath('/content/cv-cam-based-img-segmentation/data')
MODEL_DIR=PosixPath('/content/drive/MyDrive/Colab Notebooks/models')
OUTPUT_DIR=PosixPath('/content/drive/MyDrive/Colab Notebooks/output')


In [4]:
import torch

import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from tqdm import tqdm
from src import utils
from src.models.baseline_unet import UNet

In [5]:
# Hyperparameters
IMAGE_SIZE = 256
NUM_CLASSES = 2

TRAIN_EPOCHS = 20
TRAIN_BATCH_SIZE = 32
TEST_BATCH_SIZE = 32
NUM_WORKERS = 4

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pin_memory = True if torch.cuda.is_available() else False
print(f"Using device: {device}")

# load dataset
train_transforms = utils.Compose([
    utils.PILToTensor(),
    utils.ResizeImgAndMask(size=(IMAGE_SIZE, IMAGE_SIZE)),
    utils.ConvertMaskToBinary(),
    utils.RandomHorizontalFlip(flip_prob=0.5),
    utils.RandomVerticalFlip(flip_prob=0.5),
    utils.RandomRotation(degrees=30),
    utils.ToDtype(dtype=torch.float32, scale=True),
])

test_transforms = utils.Compose([
    utils.PILToTensor(),
    utils.ResizeImgAndMask(size=(IMAGE_SIZE, IMAGE_SIZE)),
    utils.ConvertMaskToBinary(),
    utils.ToDtype(dtype=torch.float32, scale=True),
])

trainset, testset = utils.construct_dataset(
    data_dir=DATA_DIR,
    train_transforms=train_transforms,
    test_transforms=test_transforms,
)

trainloader = DataLoader(
    trainset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=pin_memory,
)

testloader = DataLoader(
    testset,
    batch_size=TEST_BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=pin_memory,
)

model = UNet(num_classes=2).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(
    model.parameters(), lr=0.01,
    momentum=0.9,
    weight_decay=1e-4,
    nesterov=True,
)

Using device: cuda


100%|██████████| 792M/792M [00:45<00:00, 17.5MB/s]
100%|██████████| 19.2M/19.2M [00:02<00:00, 7.88MB/s]


In [7]:
# Train model
model.train()
num_train_samples = len(trainloader.dataset)

for epoch in range(TRAIN_EPOCHS):
    print(f"Epoch {epoch + 1}/{TRAIN_EPOCHS}")
    miou = 0
    for i, (images, segs, _, _) in enumerate(tqdm(trainloader)):
        images = images.to(device)
        segs = segs.to(device)
        batch_size = images.size(0)

        # Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, segs)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pred = torch.argmax(outputs, dim=1)
        miou += utils.mean_iou(pred, segs, num_classes=NUM_CLASSES) * batch_size

    miou /= num_train_samples
    print(f"Epoch [{epoch + 1}/{TRAIN_EPOCHS}], Loss: {loss.item():.4f}, mIoU: {miou:.4f}")

    # Save model checkpoint
    if (epoch + 1) % 5 == 0:
        save_path = MODEL_DIR.joinpath(f"baseline_unet_epoch_{epoch + 1}.pth")
        torch.save(model.state_dict(), save_path)
        print(f"Model saved at {save_path}")


Epoch 1/20


100%|██████████| 161/161 [00:35<00:00,  4.49it/s]


Epoch [1/20], Loss: 0.5379, mIoU: 0.5731
Epoch 2/20


100%|██████████| 161/161 [00:34<00:00,  4.68it/s]


Epoch [2/20], Loss: 0.3339, mIoU: 0.7146
Epoch 3/20


...

Epoch [18/20], Loss: 0.1460, mIoU: 0.8652
Epoch 19/20


100%|██████████| 161/161 [00:34<00:00,  4.68it/s]


Epoch [19/20], Loss: 0.2221, mIoU: 0.8668
Epoch 20/20


100%|██████████| 161/161 [00:34<00:00,  4.68it/s]


Epoch [20/20], Loss: 0.1328, mIoU: 0.8706
Model saved at /content/drive/MyDrive/Colab Notebooks/models/baseline_unet_epoch_20.pth


In [8]:
save_epochs = torch.arange(5, TRAIN_EPOCHS + 1, 5)

for epoch in save_epochs:
    model_name = f"baseline_unet_epoch_{epoch}.pth"

    # Load model
    model = UNet(num_classes=2)
    model.load_state_dict(torch.load(MODEL_DIR.joinpath(model_name), weights_only=True))
    model.to(device)

    # Evaluate model
    model.eval()
    num_test_samples = len(testloader.dataset)
    miou = 0
    with torch.no_grad():
        for images, segs, _, _ in tqdm(testloader):
            images = images.to(device)
            segs = segs.to(device)
            batch_size = images.size(0)

            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)

            miou += utils.mean_iou(preds, segs, num_classes=NUM_CLASSES) * batch_size

        miou /= num_test_samples
        print(f"Model: {model_name},Mean IoU: {miou:.4f}")


100%|██████████| 69/69 [00:05<00:00, 12.02it/s]


Model: baseline_unet_epoch_5.pth,Mean IoU: 0.7920


100%|██████████| 69/69 [00:06<00:00, 10.61it/s]


Model: baseline_unet_epoch_10.pth,Mean IoU: 0.8290


100%|██████████| 69/69 [00:05<00:00, 11.96it/s]


Model: baseline_unet_epoch_15.pth,Mean IoU: 0.8533


100%|██████████| 69/69 [00:05<00:00, 11.52it/s]

Model: baseline_unet_epoch_20.pth,Mean IoU: 0.8656
